# Exploring and Validating code for the improc module

In [ ]:
%matplotlib inline
import os
import numpy as np
import timeit
import sys
import matplotlib.pyplot as plt
from IPython.display import Image
from scipy import ndimage
from sklearn.preprocessing import Imputer

In [ ]:
sys.path.append(os.path.join(sys.path[0], '..'))
import improc

In [ ]:
test_path = "../testing/IMG_2114.PNG"
Image(filename=test_path)

In [ ]:
combined_image = ndimage.imread(test_path)
combined_image.shape

In [ ]:
def split(image):
    """Take a color/depth image pair and split out the two parts."""
    split_height = image.shape[0] / 2
    return image[:split_height], image[split_height:]
camera_image, depth_image = split(combined_image)
print camera_image.shape, depth_image.shape

In [ ]:
depth_image[0].shape

# Naive python code for decoding depth (slow)

In [ ]:
def convert_to_depth(image):
    height = image.shape[0]
    width = image.shape[1]
    depths = np.zeros((height, width), dtype=np.float32)
    BYTE_MAX = 255
    CHANNEL_MAX = 8
    MAX_RED_VALUE = BYTE_MAX - CHANNEL_MAX
    CHANNELS_MAX = CHANNEL_MAX * CHANNEL_MAX
    INVALID = float('Nan')
    found_orientation = False
    orientation = [1, 0, 0, 0]
    for y in xrange(0, height):
        row = image[y]
        depth_row = depths[y]
        for x in xrange(0, width):
            pixel = row[x]
            red = pixel[0]
            if x == 0 and y == 0 and red == BYTE_MAX:
                found_orientation = True
                depth_row[x] = INVALID
                continue
            if found_orientation:
                for c in range(0, len(orientation)):
                    orientation[c] = ((2.0 * pixel[c]) / BYTE_MAX) - 1
                found_orientation = False
                depth_row[x] = INVALID
                continue
            if red == 0:
                depth_row[x] = INVALID
                continue
            green = pixel[1] - red
            blue = pixel[2] - red
            depth_row[x] = (MAX_RED_VALUE - red)*CHANNELS_MAX + green*CHANNEL_MAX + blue 
    return depths, orientation

In [ ]:
depths, attitude = convert_to_depth(depth_image)

In [ ]:
depths.shape

In [ ]:
attitude

In [ ]:
depths[0][0:4]

In [ ]:
depths[120][300:310]

In [ ]:
timeit.timeit(
    "depths, attitude = convert_to_depth(depth_image)",
    number=10,
    setup="from __main__ import convert_to_depth, depth_image"
)

# Numpy rewrite of decoding (much faster)

In [ ]:
def decode_depth(image):
    BYTE_MAX = 255
    CHANNEL_MAX = 8.0
    MAX_RED_VALUE = BYTE_MAX - CHANNEL_MAX
    CHANNELS_MAX = CHANNEL_MAX * CHANNEL_MAX
    orientation = [1, 0, 0, 0]
    
    if np.array_equal(image[0, 0], [BYTE_MAX, 0, 0, BYTE_MAX]):
        print "Found attitude"
        pixel = image[0, 1]
        for c in range(len(orientation)):
            orientation[c] = ((2.0 * pixel[c]) / BYTE_MAX) - 1
        image[0, 0] = [0, 0, 0, BYTE_MAX]
        image[0, 1] = [0, 0, 0, BYTE_MAX]

    red = image[:, :, 0]
    green = image[:, :, 1] - red
    blue = image[:, :, 2] - red

    depth = (MAX_RED_VALUE - red)*CHANNELS_MAX + green*CHANNEL_MAX + blue
    depth[np.where(red == 0)] = np.nan
    return depth, orientation

In [ ]:
depths, attitude = decode_depth(depth_image)

In [ ]:
depths.shape

In [ ]:
depths[120][300:310]

In [ ]:
attitude

In [ ]:
timeit.timeit(
    "depths, attitude = decode_depth(depth_image)",
    number=10,
    setup="from __main__ import decode_depth, depth_image"
)

In [ ]:
plt.imshow(depths)

# Filling holes with sklearn's Imputer
Replaces NaN with mean/max of row or column

In [ ]:
imp = Imputer(strategy="mean")
cleaned = imp.fit_transform(depths)

In [ ]:
plt.imshow(cleaned)

In [ ]:
depths[330,0]

In [ ]:
cleaned[330,0]

# Filling holes with block mean
Replace NaN with mean/max of corresponding 15x20 pixel blocks

In [ ]:
reshaped = depths.copy().reshape(32, 15, 32, 20)
masked = np.ma.masked_array(reshaped, np.isnan(reshaped))
masked_means = np.mean(np.mean(masked, axis=3), axis=1)
means = masked_means.filled(np.nan)
plt.imshow(means)

In [ ]:
expanded = np.repeat(means,20,axis=1).reshape(32,640)
expanded = np.repeat(expanded,15,axis=0).reshape(480,640)
print expanded.shape
plt.imshow(expanded)

In [ ]:
ind = np.where(np.isnan(depths))
cleaned = depths.copy()
cleaned[ind] = expanded[ind]

In [ ]:
plt.imshow(cleaned)

# Fill NaNs with localized stat values using mipmaps
Combined this: http://stackoverflow.com/questions/14549696/mipmap-of-image-in-numpy

With this: http://stackoverflow.com/questions/5480694/numpy-calculate-averages-with-nans-removed

In [ ]:
def mipmap_imputer(image, strategy=np.mean, scales=None):
    scales = scales if scales else [(5,5), (3,2), (2,2), (2,2), (2,2), (2,2), (2,2), (1,2)]
    mipmaps = []
    mipmap = image
    for y, x in scales:
        mipmap = mipmap.copy()
        size = mipmap.shape
        reshaped = mipmap.reshape(size[0] / y, y, size[1] / x, x)
        masked = np.ma.masked_array(reshaped, np.isnan(reshaped))
        mipmap = strategy(strategy(masked, axis=3), axis=1).filled(np.nan)
        mipmaps.append(mipmap)
    
    for index, mipmap in reversed(list(enumerate(mipmaps))):
        y, x = scales[index]
        expanded = mipmap
        if x > 1:
            x_size = expanded.shape[1] * x
            expanded = np.repeat(expanded, x, axis=1).reshape(expanded.shape[0], x_size)
        if y > 1:
            y_size = expanded.shape[0] * y
            expanded = np.repeat(expanded, y, axis=0).reshape(y_size, expanded.shape[1])
        target = mipmaps[index - 1] if index > 0 else image.copy()

        nans = np.where(np.isnan(target))
        target[nans] = expanded[nans]
    return target

In [ ]:
cleaned = mipmap_imputer(depths)
plt.imshow(cleaned)

In [ ]:
timeit.timeit(
    "cleaned = mipmap_imputer(depths)",
    number=10,
    setup="from __main__ import mipmap_imputer,depths"
)

In [ ]:
_, poor_depth = split(ndimage.imread("../testing/IMG_3410.PNG"))
poor_depths, _ = decode_depth(poor_depth)
plt.imshow(poor_depths)

In [ ]:
plt.imshow(mipmap_imputer(poor_depths, strategy=np.max))

# Add Smoothing

In [ ]:
smoothed = improc.mipmap_imputer(depths, smooth=True)
plt.imshow(smoothed)

In [ ]:
poor_smoothed = improc.mipmap_imputer(poor_depths, smooth=True)
plt.imshow(poor_smoothed)